In [ ]:
%pip install beautifulsoup4

In [ ]:
%pip install tinydb

Define a function for retrieving a page of filtered articles

In [ ]:
import requests

HTTP_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0'
}

def retrieve_page(page_number):
    url = 'https://midasnetwork.us/wp-admin/admin-ajax.php'

    data = {
        'action': 'filter_papers',
        'paged': f'{page_number}',
        'journal': '',
        'author': '',
        'title': '',
        'startdate': '2024-11-01',
        'enddate': '',
        'displaydefault': '',
    }

    return requests.post(url, data=data, headers=HTTP_HEADERS)

Retrieve the first page of the paginated list of filtered articles. Get the number of the last page.

In [ ]:
from bs4 import BeautifulSoup

page = retrieve_page(1)

soup = BeautifulSoup(page.text, 'html.parser')
pages = soup.find_all('a', {'class', 'page-numbers'})

last_page = pages[-2].text

print(f'pages to process: {last_page}')

Create an array of links to each articles detail page.

In [ ]:
article_links = []
i = 2

while True:
    print(f'processing page {i-1}')

    for article in soup.find_all('article'):
        link = article.find('a')
        article_links.append(link.get('href'))

    # stop if we just processed the last page
    if i == last_page:
        break

    print(f'retrieving page {i}')

    page = retrieve_page(i)
    soup = BeautifulSoup(page.text, 'html.parser')

    i += 1

In [ ]:
import json

articles = []

for link in article_links:
    resp = requests.get(link, headers=HTTP_HEADERS)
    soup = BeautifulSoup(resp.text, 'html.parser')

    data = soup.find_all('p', {'class', 'elementor-heading-title'})
    
    article = {
        'title': data[0].text,
        'abstract': data[1].text,
        'journal': data[2].text,
        'reference': data[3].text
    }

    refs = data[3].find_all('a')
    article['link'] = refs[1].get('href')

    articles.append(article)

In [ ]:
from tinydb import TinyDB, Query

db = TinyDB('db.json')
table = db.table('name')

for article in articles:
    table.insert(article)